<a href="https://colab.research.google.com/github/SantiagoTadini/TP2-Datos/blob/main/ModeloXGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import log_loss
import pandas as pd
import datetime as dt
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from urllib.request import urlretrieve


from sklearn.model_selection import train_test_split

In [102]:
url_train = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_train.csv'
url_test = 'https://raw.githubusercontent.com/SantiagoTadini/TP2-Datos/main/Features/data_test.csv'

data_train = pd.read_csv(url_train) #se usa para entrenar el modelo
data_test = pd.read_csv(url_test) #se usa para predecir

In [103]:
#Separo el data frame de train en uno para entrenar y uno para probar.
model_train, model_test = train_test_split(data_train, test_size=.2, random_state=1999)


In [104]:
 #Quitamos la variable target de los datos usando el método drop() y creamos dos matrices, una para nuestro set de entrenamiento y otra para el set de prueba.

model_train_mat = xgb.DMatrix(model_train.drop("Target", 1), label=model_train["Target"])
model_test_mat = xgb.DMatrix(model_test.drop("Target", 1), label=model_test["Target"])

model_train_mat


In [105]:
#Podemos comenzar con el entrenamiento del modelo predictivo.
#Para entrenar un modelo usamos la función train() de xgboost.

#Hay que definir hiperparametros. Por ahora pongo medio random y despues vemos
parametros = {"booster":"gbtree", "max_depth": 4, "eta": .2, "objective": "binary:logistic", "nthread":2}
rondas = 100

#definimos una variable que contiene una lista de tuplas con los sets que serán usados para realizar la validación del modelo después de cada ronda.
evaluacion = [(model_test_mat, "eval"), (model_train_mat, "train")]

In [106]:
modelo = xgb.train(parametros, model_test_mat, rondas, evaluacion)


[0]	eval-error:0.130589	train-error:0.142011
[1]	eval-error:0.124963	train-error:0.135347
[2]	eval-error:0.114007	train-error:0.130238
[3]	eval-error:0.111638	train-error:0.129424
[4]	eval-error:0.109861	train-error:0.126981
[5]	eval-error:0.106603	train-error:0.124389
[6]	eval-error:0.104827	train-error:0.12313
[7]	eval-error:0.105123	train-error:0.123871
[8]	eval-error:0.100977	train-error:0.118836
[9]	eval-error:0.098904	train-error:0.118244
[10]	eval-error:0.100681	train-error:0.118022
[11]	eval-error:0.100385	train-error:0.117725
[12]	eval-error:0.099497	train-error:0.117429
[13]	eval-error:0.099497	train-error:0.117133
[14]	eval-error:0.097424	train-error:0.117503
[15]	eval-error:0.096535	train-error:0.116541
[16]	eval-error:0.096832	train-error:0.116837
[17]	eval-error:0.095943	train-error:0.116615
[18]	eval-error:0.096535	train-error:0.116171
[19]	eval-error:0.09772	train-error:0.116245
[20]	eval-error:0.095943	train-error:0.115282
[21]	eval-error:0.095647	train-error:0.113875


In [107]:
modelo

In [108]:
#El siguiente paso es utilizar el método predict() de nuestro modelo con los datos de prueba para generar predicciones.
prediccion = modelo.predict(model_test_mat)
prediccion

array([1.7109717e-01, 4.3782013e-04, 1.9106077e-03, ..., 8.8751829e-01,
       9.7931206e-01, 1.2211549e-02], dtype=float32)

In [109]:
#Evaluacion del modelo:
def metricas(objetivo, prediccion):
    matriz_conf = confusion_matrix(objetivo, prediccion)
    score = accuracy_score(objetivo, prediccion)
    reporte = classification_report(objetivo, prediccion)
    metricas = [matriz_conf, score, reporte]
    return(metricas)

#si >0.5, es 1 (won). si <0.5, es 0 (lost)
prediccion_bin = [1 if i > .5 else 0 for i in prediccion]
prediccion_bin[:10]

metrics = metricas(model_test["Target"], prediccion_bin)
[print(i) for i in metrics]


[[1372   95]
 [  19 1891]]
0.966242226828546
              precision    recall  f1-score   support

           0       0.99      0.94      0.96      1467
           1       0.95      0.99      0.97      1910

    accuracy                           0.97      3377
   macro avg       0.97      0.96      0.97      3377
weighted avg       0.97      0.97      0.97      3377



[None, None, None]

In [110]:
error = log_loss(model_test['Target'], prediccion)
error

0.11943075546697057

In [111]:
#---------------------------------------------------------------------------------------------------------------

In [ ]:
#Hasta ahi estuve jugando, ahora empieza la posta: Hay que agrupar por categoria.

# Supongo que el DataFrame con el que trabajo se llama 'data_train'
# 'x' son los datos (todas las columnas menos el target), 'y' es el target
x, y = data_train.iloc[:, :-1], data_train.iloc[:, -1]

# 'x' son los datos --> x_train son los datos con los que entrenar
#                   --> x_test son los datos con los que probar
# 'y' es el target  --> y_train son las soluciones con las que comparar lo entrenado con x_train
#                   --> y_test son las soluciones con las que comparar lo probado con x_test

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)
x_train

#Esto lo hizo chechu y esta bien, no cambiar.

In [206]:
 xg_reg = xgb.XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 7 , alpha = 10, n_estimators = 10, eta=0.2,nthread=2,objective='binary:logistic',booster='gbtree')
 xg_reg.fit(x_train, y_train)
 preds = xg_reg.predict(x_test)



In [207]:
error = log_loss(model_test['Target'], preds)
error

0.783968855307799

In [148]:
# Sacamos el y_test_de_practica_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

stage_test_de_practica = y_test.to_frame()

oportunidades_test_de_practica['Target'] = stage_test_de_practica
y_test_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
y_test_agrupado_por_opp = y_test_agrupado_por_opp.reset_index()

In [208]:
# Sacamos el preds_agrupado_por_opp
oportunidades_test_de_practica = x_test['Opportunity_ID']
oportunidades_test_de_practica = oportunidades_test_de_practica.to_frame()

oportunidades_test_de_practica['Target'] = preds

preds_agrupado_por_opp = oportunidades_test_de_practica.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_agrupado_por_opp = preds_agrupado_por_opp.reset_index()

In [209]:
# Calculamos el error con groupby hecho
error = log_loss(y_test_agrupado_por_opp['Target'], np.nan_to_num(preds_agrupado_por_opp['Target'].to_numpy()))
error

0.4416105265845752

In [210]:
# Ahora predecimos con el set de test
preds_test = xg_reg.predict(data_test)

In [212]:
# Sacamos el preds_test_agrupado_por_opp
oportunidades_test = data_test['Opportunity_ID']
oportunidades_test = oportunidades_test.to_frame()

oportunidades_test['Target'] = preds_test

preds_test_agrupado_por_opp = oportunidades_test.groupby('Opportunity_ID').agg({'Target' : 'mean'})
preds_test_agrupado_por_opp = preds_test_agrupado_por_opp.reset_index()

In [214]:
from google.colab import files
preds_test_agrupado_por_opp.to_csv('preds_test.csv', index=False, header=True)
files.download('preds_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [213]:
preds_test_agrupado_por_opp

,Opportunity_ID,Target
0,10689,0.600012
1,10690,0.578072
2,10691,0.530447
3,10692,0.529289
4,10693,0.720677
...,...,...
1562,12364,0.663277
1563,12365,0.449136
1564,12366,0.378946
1565,12367,0.373662
